In [1]:
import numpy as np

# Best time to buy and sell stock (Easy)

### Description
You are given an array prices where prices[i] is the price of a given stock on the ith day.

You want to maximize your profit by choosing a single day to buy one stock and choosing a different day in the future to sell that stock.

Return the maximum profit you can achieve from this transaction. If you cannot achieve any profit, return 0.
##### Example:
Input: prices = [7,1,5,3,6,4]

Output: 5 

Explanation: Buy on day 2 (price = 1) and sell on day 5 (price = 6), profit = 6-1 = 5.

Note that buying on day 2 and selling on day 1 is not allowed because you must buy before you sell.


### Analysis

Suppose we are the trader. 
If we want to buy the asset, we will choose a day on which its price is minimal. This obvious fact gives us a first interesting point: on the date $i$ on which we buy the asset, $prices[i]$ is a local minimum (ie: $prices[i+1] \geq prices[i]$ and $prices[i-1] \geq prices[i]$).
On the other hand and using the same idea, since we aim to sell the asset with the most profit possible, if it was purchased at the date $i$, we will $\textbf{choose as selling date the day the stock is at its maximum (after day $i$)}$. 

Hence, there exists a day $i_{0}$ for which we can write:  $maxprofit = \text{maxPriceAfterDay}i_{0} - prices[i_{0}]$ with $maxprofit$ being the best transaction's profit (ie what we are looking for) and $\text{maxPriceAfterDay}i_{0}$ the maximum value of $prices[j]$  for  $j \geq i$.

We then only need to compare the different values of $\text{maxPriceAfterDay}i - prices[i]$  for $i$ in range(len(prices)) and choose the maximum value.




### Codes

#### Implementation 1: non vectorialized - O(N) time, O(1) extra space 

Here is a first implementation of our analysis, without using numpy and vectorialization. Although the use of a for loop is slower than using a numpy function, here it is still interesting because it allows us to use only two additional variables, resulting in a constant extra space algorithm. 


In [ ]:
def maxProfit(prices):
        """
        :type prices: List[int]
        :rtype: int
        """       
        n = len(prices)
        maximum = prices[-1]    #maximum will store the maximal value of prices[i] for indices after i. We begin here with the end of the array, i = n-1, and will use a decreasing loop 
        maxpro = 0              #The maximum profit we will return
        for i in range(n-1,-1,-1):
          if maximum - prices[i] > maxpro :  #Update the profit when we find a better option
            maxpro = maximum - prices[i]
          if prices[i] > maximum: 
            maximum = prices[i]     #Update the current maximum when we find a larger element in the prices array
        return maxpro

#### Implementation 2: using numpy functions- O(N) time, O(N) extra space
Since for loops can be time consuming, here I present an implementation which allows us to avoid using them.
This solution is more straighforward, but the backlash is that the moving maximum we used in the implementation 1 is now replaced by an array. While in time this solution is better, in space it is not good at all.

By using numpy.maximum.accumulate() and numpy.flip(), we define an array (called "maximum" in the following cell) which contains for every index $i$ the maximal value of $prices[j]$ for $j \geq i$.

Since we know that there exists a day $i_{0}$ for which we can write:  $maxpro = \text{maxPriceAfterDay}i_{0} - prices[i_{0}] = maximum[i_{0}] - prices[i_{0}]$, we just have to return the max value of the array "maximum - prices" 

In [ ]:
def maxProfit(prices):
      """
      :type prices: List[int]
      :rtype: int
      """    
      maximum = np.maximum.accumulate(np.flip(prices)) #for every i maximum[n-1-i] is the maximal value of prices[j] for j>=i
      maximum = np.flip(maximum)                       #for every i maximum[i] is now the maximal value of prices[j] for j>=i
      maximum = maximum - prices
      return np.max(maximum)

# Best time to buy and sell stock II (medium)

### Description
You are given an integer array prices where $prices[i]$ is the price of a given stock on the ith day.

On each day, you may decide to buy and/or sell the stock. You can only hold at most one share of the stock at any time. However, you can buy it then immediately sell it on the same day.

Find and return the maximum profit you can achieve.
##### Example:
Input: prices = [7,1,5,3,6,4]
Output: 7
Explanation: Buy on day 2 (price = 1) and sell on day 3 (price = 5), profit = 5-1 = 4.
Then buy on day 4 (price = 3) and sell on day 5 (price = 6), profit = 6-3 = 3.
Total profit is 4 + 3 = 7.



### Analysis
Here we begin buy noticing the fact that, having bought an asset, if its price is increasing we will not sell it (the optimal time has not come yet). We will wait until the price stops increasing, and then we can sell it. 

Similarly, when the stock's price is decreasing, we do not want to buy it, since waiting ensures us a better price. We will wait until it stops decreasing, and then we can buy it (it stops decreasing when it has reached a local minimum) .
This gives us the general idea of the algorithm presented in the cell below. 

We introduce the boolean "purchased_stock" which encodes the information of whether we have a stock at hand or not. 

### Implementation - O(n) time - O(1) space

In [ ]:
def maxProfit(prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        purchased_stock = False #gives us the state of the trader. True if he has purchased a stock and is looking to sell it, False otherwise. 
        max_profit = 0         #result we return
        purchase_date = 0       #stores the purchase date of the current stock of the trader (if he has one). Helps us when we sell a stock, to update the sum of our profits
        for i in range(len(prices)):
          if i+1 < len(prices) and prices[i]<=prices[i+1]: # if stock's price is increasing
            if not purchased_stock : # if the trader does not already have the stock, he has to buy it
              purchase_date = i
              purchased_stock = True
          else: #stock's price has stopped growing
            if purchased_stock: #if the trader has the stock, he needs to sell it and wait for better times to come 
              max_profit = max_profit + prices[i] - prices[purchase_date]
              purchased_stock = False
        return max_profit

### Vectorialization ?
Here my attempts of vectorialization have failed because of the many edge cases to deal with.

The idea was to rethink at the problem as one that consists to find local maximums and local minimums of the array prices and store them in respective arrays localMax and localMin, with the guess that:  $maxProfit = \sum (localMax[i] - localMin[i])$ since we only buy at local minimums and sell at local maximums (to find those local mins and max, "argrelextrema" from scipy.signal can be suitable). 

But the many different dispositions of those local maximums and minimums make the choice of the variables $i$ in the sum not easy. We can think of the case when the first element $prices[0]$ is a local maximum, being a useless local maximum (and then should not appear in the sum).

Furthermore, a vectorialized implementation here will have the same backlash as the vectorialized implementation of the first problem, which is the use of O(n) space. Vectorialization is not that interesting in this case.  

# Best time to buy and sell stock III (hard)

### Description
You are given an array prices where prices[i] is the price of a given stock on the ith day.

Find the maximum profit you can achieve. You may complete at most two transactions.

Note: You may not engage in multiple transactions simultaneously (i.e., you must sell the stock before you buy again).

 

#### Example 1:

Input: prices = [3,3,5,0,0,3,1,4]
Output: 6
Explanation: Buy on day 4 (price = 0) and sell on day 6 (price = 3), profit = 3-0 = 3.
Then buy on day 7 (price = 1) and sell on day 8 (price = 4), profit = 4-1 = 3.

### Analysis
Suppose we have a solution and the corresponding indices in which we buy and sell:
$(i_{0},i_{1},j_{0},j_{1})$ such that we buy the stock at $i_{0}$ and sell it at $i_{1}$, then buy it again at $j_{0}$ and sell it at $j_{1}$ (without any loose we suppose $i_{0} \leq j_{0}$. Let's look at some of the properties of our solution.

First of all, since we cannot engage in multiple transactions, the two transactions we have here do not overlap. (this gives us $i_{1} \leq j_{0}).$

Let's have a closer look to $j_{0}$ (the date at which we purchase the asset for a second time).

Since the solution at hand is optimal, buying the asset at $i_{0}$ and selling it at $i_{1}$ was the best possible move before the date $j_{0}$. If this was not true, it would have been possible to find two indices $k$ and $l$ such that buying at date $k$ and selling at date $l$ rewards more than buying at $i_{0}$ and selling at $i_{1}$. 
Then, the quadruplet $(k,l,j_{0},j_{1})$ would have been a better solution than the quadruplet $(i_{0},i_{1},j_{0},j_{1})$, this is absurd given our hypothesis on $(i_{0},i_{1},j_{0},j_{1})$.

We conclude that in our algorithm,we have to find the best transaction before $j_{0}$.

The same point stands when we look at buying at the date $j_{0}$ and selling at the date $j_{1}$. We have to perform the best possible transaction, but in this case since we already know that we purchase at date $j_{0}$, the best transaction just consists of selling the stock when its price is the maximum of the prices after the date $j_{0}$.

Then the following equation is true for the index $j_{0}$:


result $=$ maximum_profit $=$ (profit_of_best_transaction_before$(j_{0})$) + (maximum_price_after($j_{0}$) - prices$[j_{0}])$

The first term is the best transaction before $j_{0}$ and the second is the best transaction after $j_{0}$.

This gives us the idea of our algorithm. Since we do not know $j_{0}$ (we only know that such $j_{0}$ exists), we will go through all the indices in the array prices, computing the best profit for a single transaction before index $i$ and adding it to the best profit for a single transaction which starts at $i$.


To be efficient, we keep track of the best profit before index $i$ while moving in the array with the relation:

best_profit_before_i+1 = max (best_profit_before_i,  price_i - minimal_price_before_i)
since we have two options: either the best profit before i is obtained with $i_{1} < j_{0}$ (first case in the max, the sell date of the best transaction before $j_{0}$  is stricly before $j_{0}$ ) or it is obtained with $i_{1} = j_{0}$ (the sell date of the best transaction before $j_{0}$  is $j_{0}$).

### Implementation

#### Implementation 1: no vectorialization - O(N) time - O(N) space

In [ ]:
def maxProfit(prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        n = len(prices)
        best_profit_previous = [0]*n     # for every i best_profit_previous[i] will be the best profit possible for a single transaction before the date i 
        minimum_previous = [0]*n         # for every i minimum_previous[i] will be the minimum price of the stock before the date i
        maximum_after = [0]*n            # for every i maximum_after[i] will be the maximum price of the stock after the date i
        minimum_previous[0] = prices[0]
        best_profit_previous[0] = 0
        maximum_after[-1] = prices[-1]
        maxpro = 0                       #Our final result, the max profit
        for i in range(1,n):             #filling the minimum and maximum tabs
            minimum_previous[i] = min(minimum_previous[i-1],prices[i])
            maximum_after[n - i -1] = max(maximum_after[n - i],prices[n - i -1])
        for i in range(1,len(prices)):   #We try all the possible j_0 of our analysis(see the cell above), taking the maximum profit achieved
            best_profit_previous[i] = max(best_profit_previous[i-1],prices[i] - minimum_previous[i]) #the sell date of the best single transaction is either before date i or exactly at date i
            maxpro = max(maxpro,best_profit_previous[i] + maximum_after[i] - prices[i])
        return maxpro

### Implementation 2: using numpy's functions accumulate - faster

Unlike the second problem, the vectorialization of this one is straighforward after having written the first implementation. 

The idea of the algorithm is exactly the same, (while this one is much less readable) and here we end up with a solution using the same order of extra space as the previous, but faster. (I have written it while writing this notebook and it ended up beating 98% of the other python 3 solutions in time). 

I still use the function numpy's accumulate which here has the exact same use as the tabs "minimum_previous" and "maximum after"


In [ ]:
def maxProfit(prices):
      """
      :type prices: List[int]
      :rtype: int
      """
      best_profit_previous = np.maximum.accumulate(prices - np.minimum.accumulate(prices))
      reversed_prices = prices[::-1] #I have to reverse since accumulate does not have a parameter to tell if we what the index to be increasing or decreasing
      best_profit_after = np.maximum.accumulate(np.maximum.accumulate(reversed_prices) - reversed_prices)[::-1]
      return np.max(best_profit_after + best_profit_previous)